**Installation Of Require Libraries**

In [ ]:
!pip install ultralytics
!pip install opencv-python-headless




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 532.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:00


**Code for detecting the Pedestrians witch is in the middle of the road**

In [ ]:
# Install the ultralytics package
# !pip install ultralytics

import cv2
import torch
from ultralytics import YOLO
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab
from IPython.display import display, Video
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load YOLOv8 model
model = YOLO('yolov8s.pt')  # You can choose 'yolov8m.pt', 'yolov8l.pt', etc.

# Initialize video capture
input_video_path = '/content/drive/MyDrive/Colab Notebooks/Pedestrian detection/Realclip.mp4'
cap = cv2.VideoCapture(input_video_path)

# Get the video details for output
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define output path within Google Drive
output_video_path = '/content/drive/MyDrive/Colab Notebooks/Pedestrian detection/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))  # Output file

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection on the full frame
    results = model(frame)

    # Extract information
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # x1, y1, x2, y2
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy().astype(int)

        for box, confidence, class_id in zip(boxes, confidences, class_ids):
            label = model.names[class_id]
            if label == 'person' and confidence > 0.5:  # Confidence threshold and class check
                x1, y1, x2, y2 = map(int, box)

                # Draw bounding box and label
                color = (255, 0, 0)  # Blue color for bounding boxes
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release everything when done
cap.release()
out.release()

# Display the output video
display(Video(output_video_path, embed=True))

print(f"Output video saved to {output_video_path}")


Streaming output truncated to the last 5000 lines.
0: 384x640 3 persons, 14 cars, 2 motorcycles, 1 truck, 408.5ms
Speed: 4.0ms preprocess, 408.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 14 cars, 2 motorcycles, 2 trucks, 403.4ms
Speed: 3.3ms preprocess, 403.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 13 cars, 1 motorcycle, 3 trucks, 398.8ms
Speed: 3.7ms preprocess, 398.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 13 cars, 3 motorcycles, 1 truck, 409.9ms
Speed: 3.7ms preprocess, 409.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 15 cars, 1 motorcycle, 1 truck, 402.2ms
Speed: 3.4ms preprocess, 402.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 14 cars, 2 trucks, 449.9ms
Speed: 3.6ms preprocess, 449.9ms inference, 1.8ms postprocess per image at shape (1, 3, 38

**Impove the code wich is solving the problem of detecting the people wich is riding the bike**

In [ ]:
# Install the ultralytics package
# !pip install ultralytics

import cv2
import torch
from ultralytics import YOLO
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab
from IPython.display import display, Video
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load YOLOv8 model
model = YOLO('yolov8s.pt')  # You can choose 'yolov8m.pt', 'yolov8l.pt', etc.

# Initialize video capture
input_video_path = '/content/drive/MyDrive/Colab Notebooks/Pedestrian detection /Realclip.mp4'
cap = cv2.VideoCapture(input_video_path)

# Get the video details for output
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define output path within Google Drive
output_video_path = '/content/drive/MyDrive/Colab Notebooks/Pedestrian detection /output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))  # Output file

def is_overlapping(box1, box2):
    """Check if two bounding boxes overlap"""
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Determine the coordinates of the intersection rectangle
    x_min = max(x1_min, x2_min)
    y_min = max(y1_min, y2_min)
    x_max = min(x1_max, x2_max)
    y_max = min(y1_max, y2_max)

    # Compute the area of the intersection rectangle
    overlap_width = max(0, x_max - x_min)
    overlap_height = max(0, y_max - y_min)
    overlap_area = overlap_width * overlap_height

    # If there's no overlap, the area will be zero
    if overlap_area > 0:
        return True
    return False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection on the full frame
    results = model(frame)

    # Extract information
    person_boxes = []
    bicycle_boxes = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # x1, y1, x2, y2
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy().astype(int)

        for box, confidence, class_id in zip(boxes, confidences, class_ids):
            label = model.names[class_id]
            if label == 'person' and confidence > 0.5:
                person_boxes.append(box)
            elif label == 'bicycle' and confidence > 0.5:
                bicycle_boxes.append(box)

    # Draw bounding boxes only for non-overlapping persons
    for person_box in person_boxes:
        overlap = False
        for bicycle_box in bicycle_boxes:
            if is_overlapping(person_box, bicycle_box):
                overlap = True
                break

        if not overlap:
            x1, y1, x2, y2 = map(int, person_box)
            color = (255, 0, 0)  # Blue color for bounding boxes
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, 'person', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release everything when done
cap.release()
out.release()

# Display the output video
display(Video(output_video_path, embed=True))

print(f"Output video saved to {output_video_path}")


Streaming output truncated to the last 5000 lines.
0: 384x640 7 persons, 14 cars, 1 motorcycle, 1 bus, 3 trucks, 65.4ms
Speed: 1.9ms preprocess, 65.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 16 cars, 2 motorcycles, 1 bus, 2 trucks, 63.2ms
Speed: 1.7ms preprocess, 63.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 16 cars, 1 motorcycle, 2 buss, 3 trucks, 64.0ms
Speed: 1.6ms preprocess, 64.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 17 cars, 1 motorcycle, 1 bus, 3 trucks, 65.5ms
Speed: 1.6ms preprocess, 65.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 17 cars, 3 motorcycles, 1 bus, 2 trucks, 65.0ms
Speed: 1.7ms preprocess, 65.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 17 cars, 3 motorcycles, 4 trucks, 69.0ms
Speed: 1.7ms preprocess, 69.0ms inference, 0.9ms